## Import libraries

In [55]:
import os
import string
from typing import Tuple

import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import pad_sequences
from keras.preprocessing import text
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM, Bidirectional

import giskard
from giskard import Dataset, Model

## Define constants

In [56]:
# Constants.
TEXT_COLUMN_NAME = "headline"
TARGET_COLUMN_NAME = "is_sarcastic"

STOPWORDS = stopwords.words('english')

MAX_TOKENS = 25000
EMBEDDING_DIM = 200
MAX_SEQUENCE_LENGTH = 200

# Paths.
DATA_DIR = os.path.join(".", "datasets", "sarcasm_classification_dataset")
EMBEDDING_FILE = os.path.join(DATA_DIR, "glove_200d.txt")

## Load data

In [57]:
def load_data(**kwargs) -> pd.DataFrame:
    """Load data."""
    print(f"Loading data...")

    df = pd.read_json(os.path.join(DATA_DIR, "Sarcasm_Headlines_Dataset_v2.json"), lines=True, **kwargs)
    df = df.drop(columns="article_link")

    print(f"Finished loading data! \nShape: {df.shape} \nColumns: {df.columns.tolist()}")

    return df

sarcasm_df = load_data(nrows=2000)

Loading data...
Finished loading data! 
Shape: (2000, 2) 
Columns: ['is_sarcastic', 'headline']


## Define text preprocessing logic

In [58]:
def clean_text(df: pd.DataFrame) -> np.ndarray:
    """Perform text-data cleaning: punctuation and stop words removal."""

    # Remove punctuation.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(
        lambda sentence: sentence.translate(str.maketrans('', '', string.punctuation)))

    # Remove stop words.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(
        lambda sentence: ' '.join([_word for _word in sentence.split() if _word.lower() not in STOPWORDS]))

    return df[TEXT_COLUMN_NAME]

## Train-test split

In [70]:
X_train, X_test, Y_train, Y_test = train_test_split(sarcasm_df[[TEXT_COLUMN_NAME]], sarcasm_df[TARGET_COLUMN_NAME], random_state=0)

## Wrap data with giskard

In [71]:
raw_data = pd.concat([X_test.copy(), Y_test.copy()], axis=1)
wrapped_data = Dataset(raw_data, name="sarcasm", target=TARGET_COLUMN_NAME, column_types={TEXT_COLUMN_NAME: "text"})

Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."


## Preprocess text

In [72]:
print(f"Cleaning train data...")
X_train = clean_text(X_train)

print(f"Cleaning test data...")
X_test = clean_text(X_test)

print(f"Finished cleaning!")

Cleaning train data...
Cleaning test data...
Finished cleaning!


In [73]:
# Fit tokenizer.
tokenizer = text.Tokenizer(num_words=MAX_TOKENS)
tokenizer.fit_on_texts(X_train)

# Tokenize train text.
tokenized_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(tokenized_train, maxlen=MAX_SEQUENCE_LENGTH)

# Tokenize test text.
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(tokenized_test, maxlen=MAX_SEQUENCE_LENGTH)

In [81]:
num_embeddings = min(MAX_TOKENS, len(tokenizer.word_index)) + 1

## Define preprocessing function

In [74]:
def preprocessing_function(df: pd.DataFrame) -> np.ndarray:
    """Preprocessing function used by giskard."""
    # Clean text.
    cleaned_text = clean_text(df)

    # Tokenize text.
    tokens = tokenizer.texts_to_sequences(cleaned_text)
    tokens_with_padding = pad_sequences(tokens, maxlen=MAX_SEQUENCE_LENGTH)

    return tokens_with_padding

## Create embeddings matrix

In [76]:
def parse_line(word: str, *arr: list) -> Tuple[str, np.ndarray]:
    """Parse line from the file with embeddings.
    The first value of the line is the word and the rest values are related glove embedding: (<word>, 0.66, 0.23, ...)."""
    return word, np.asarray(arr, dtype='float32')

def init_embeddings_matrix(embeddings_dict: dict) -> np.ndarray:
    """Initialization of the matrix, where each row is a specific embedding vector."""
    stacked_embeddings = np.stack(list(embeddings_dict.values()))
    embeddings_mean, embeddings_std, embeddings_dimension = stacked_embeddings.mean(), stacked_embeddings.std(), stacked_embeddings.shape[1]

    embeddings_matrix = np.random.normal(embeddings_mean, embeddings_std, (num_embeddings, embeddings_dimension))

    return embeddings_matrix

def get_embeddings_matrix() -> np.ndarray:
    """Create matrix, where each row is an embedding of a specific word."""
    print(f"Building embeddings matrix...")

    # Load glove embeddings.
    embeddings_dict = dict(parse_line(*line.rstrip().rsplit(' ')) for line in open(EMBEDDING_FILE))

    # Initialization of embeddings matrix.
    embeddings_matrix = init_embeddings_matrix(embeddings_dict)

    # Fill-in embeddings matrix with glove word vectors.
    for word, idx in tokenizer.word_index.items():
        if idx >= MAX_TOKENS:
            continue

        embedding_vector = embeddings_dict.get(word, None)

        if embedding_vector is not None:
            embeddings_matrix[idx] = embedding_vector

    print(f"Finished building embedding matrix!")

    return embeddings_matrix

embed_matrix = get_embeddings_matrix()

Building embeddings matrix...
Finished building embedding matrix!


In [86]:
def init_model(weights) -> Sequential:
    # Define model container.
    _model = Sequential()

    # Embedding layer.
    _model.add(Embedding(num_embeddings, output_dim=EMBEDDING_DIM, weights=[weights], input_length=EMBEDDING_DIM, trainable=True))

    # LSTM stage.
    _model.add(Bidirectional(LSTM(units=128 , recurrent_dropout=0.5, dropout=0.5)))

    # Dense stage.
    _model.add(Dense(1, activation='sigmoid'))

    # Build model.
    _model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['acc'])
    return _model

In [87]:
model = init_model(embed_matrix)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 200)          1106400   
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              336896    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,443,553
Trainable params: 1,443,553
Non-trainable params: 0
_________________________________________________________________


In [88]:
# Define hyperparameters.
epochs = 5
batch_size = 128

# Fit model.
_ = model.fit(X_train, Y_train, batch_size=batch_size , validation_data=(X_test, Y_test), epochs=epochs)

Epoch 1/5
12/12 [==============================] - 31s 2s/step - loss: 0.8942 - acc: 0.5367 - val_loss: 0.6813 - val_acc: 0.5600
Epoch 2/5
12/12 [==============================] - 23s 2s/step - loss: 0.5716 - acc: 0.7467 - val_loss: 0.6534 - val_acc: 0.6440
Epoch 3/5
12/12 [==============================] - 24s 2s/step - loss: 0.2791 - acc: 0.8853 - val_loss: 0.7597 - val_acc: 0.6780
Epoch 4/5
12/12 [==============================] - 23s 2s/step - loss: 0.0768 - acc: 0.9767 - val_loss: 0.9629 - val_acc: 0.7000
Epoch 5/5
12/12 [==============================] - 23s 2s/step - loss: 0.0186 - acc: 0.9947 - val_loss: 1.1488 - val_acc: 0.6980


In [89]:
test_metric = model.evaluate(X_test, Y_test)[1]
print(f"Test accuracy: {test_metric}")

16/16 [==============================] - 2s 98ms/step - loss: 1.1488 - acc: 0.6980
Test accuracy: 0.6980000138282776


## Wrap data and model

In [90]:
wrapped_model = Model(model,
                      model_type="classification",
                      data_preprocessing_function=preprocessing_function,
                      name="sarcasm_classification",
                      feature_names=[TEXT_COLUMN_NAME],
                      classification_threshold=0.5,
                      classification_labels=[1, 0])

## Scan model

In [95]:
wrapped_model.predict(wrapped_data)

16/16 [==============================] - 2s 105ms/step
2023-05-16 19:01:38,716 pid:10647 MainThread giskard.models.base WARNING  
Your binary classification model prediction is of the shape (500, 1). 
In Giskard we expect the shape (500, 2) for binary classification models. 
We automatically inferred the second class prediction but please make sure that 
the probability output of your model corresponds to the first label of the 
classification_labels ([1, 0]) you provided us with.
NoneType: None


ModelPredictionResults(raw=array([[6.67162687e-02, 9.33283746e-01],
       [3.59704299e-03, 9.96402979e-01],
       [8.12155426e-01, 1.87844574e-01],
       [5.31209679e-03, 9.94687915e-01],
       [7.03685954e-02, 9.29631412e-01],
       [1.60563260e-03, 9.98394370e-01],
       [4.92867175e-03, 9.95071352e-01],
       [9.99187648e-01, 8.12351704e-04],
       [1.76677329e-03, 9.98233199e-01],
       [1.99560076e-02, 9.80044007e-01],
       [1.26275659e-01, 8.73724341e-01],
       [4.45018262e-02, 9.55498159e-01],
       [5.09051792e-02, 9.49094832e-01],
       [5.42985741e-03, 9.94570136e-01],
       [9.88711836e-04, 9.99011278e-01],
       [8.21457896e-03, 9.91785407e-01],
       [3.33183892e-02, 9.66681600e-01],
       [9.97920930e-01, 2.07906961e-03],
       [9.96558070e-01, 3.44192982e-03],
       [1.88112468e-03, 9.98118877e-01],
       [5.53899584e-03, 9.94461000e-01],
       [1.01795723e-03, 9.98982072e-01],
       [1.10409048e-03, 9.98895884e-01],
       [9.41927910e-01, 5.8072

In [96]:
scanning_result = giskard.scan(wrapped_model, wrapped_data)

2023/05/16 19:01:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2023-05-16 19:01:48.222054: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'while/Placeholder_2' with dtype float and shape [?,128]
	 [[{{node while/Placeholder_2}}]]
2023-05-16 19:01:48.377449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'while/Placeholder_2' with dtype float and shape [?,128]
	 [[{{node while/Placeholder_2}}]]
2023-05-16 19:01:48.431992:

INFO:tensorflow:Assets written to: /var/folders/4q/3_bfyqnn7yv5jcjq98x2jf680000gn/T/giskard-model-fr_6he4o/data/model/assets
2023-05-16 19:02:03,485 pid:10647 MainThread tensorflow   INFO     Assets written to: /var/folders/4q/3_bfyqnn7yv5jcjq98x2jf680000gn/T/giskard-model-fr_6he4o/data/model/assets


/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


1/1 [==============================] - 1s 567ms/step
2023-05-16 19:02:47,104 pid:10647 MainThread giskard.models.base WARNING  
Your binary classification model prediction is of the shape (10, 1). 
In Giskard we expect the shape (10, 2) for binary classification models. 
We automatically inferred the second class prediction but please make sure that 
the probability output of your model corresponds to the first label of the 
classification_labels ([1, 0]) you provided us with.
NoneType: None
1/1 [==============================] - 0s 110ms/step
2023-05-16 19:02:47,270 pid:10647 MainThread giskard.models.base WARNING  
Your binary classification model prediction is of the shape (10, 1). 
In Giskard we expect the shape (10, 2) for binary classification models. 
We automatically inferred the second class prediction but please make sure that 
the probability output of your model corresponds to the first label of the 
classification_labels ([1, 0]) you provided us with.
NoneType: None
Your m

In [97]:
display(scanning_result)

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">7 issues detected</span>
 </div>
 </div>

 
 <div data-tab-target="Model_bias" class="h-12 cursor-pointer group active">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Model bias
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 5
 </span>
 
 </div>
 </div>
 
 <div data-tab-target="Robustness" class="h-12 cursor-pointer group">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Robustness
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 2
 </span>
 
 </div>
 </div>
 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
<div id="Model_bias" role="tabpanel"
 class="dark:text-white dark:bg-zinc-800 p-4 pt-4 mb-4">
 
 <div class="p-3 mt-2 bg-amber-100/40 rounded-sm w-full flex align-middle">
 <div class="text-amber-100">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="mt-0.5 w-6 h-6">
 <path stroke-linecap="round" stroke-linejoin="round"
 d="M12 9v3.75m-9.303 3.376c-.866 1.5.217 3.374 1.948 3.374h14.71c1.73 0 2.813-1.874 1.948-3.374L13.949 3.378c-.866-1.5-3.032-1.5-3.898 0L2.697 16.126zM12 15.75h.007v.008H12v-.008z" />
 </svg>
 </div>
 <p class="ml-2 my-1 text-amber-100 text-sm">
 We detected some spots in your data where the model has a tendency to make incorrect
 predictions.
 </p>
 </div>
 

 
 <div class="flex items-center space-x-1">
 <h2 class="uppercase my-4 mr-2 font-medium">Issues</h2>
 
 <span class="text-xs border rounded px-1 uppercase text-red-400 border-red-400">4
 major</span>
 
 
 <span class="text-xs border rounded px-1 uppercase text-amber-200 border-amber-200">1
 medium</span>
 
 </div>
